# DAISY - El proceso de minería de datos para la identificación de letalidad sintética
```
Título original: Data Mining Synthetic Lethality Identification Pipeline (DAISY)
Autor: Bahar Tercan
Fecha de creación: 02-07-2022
Cuaderno de Jupyter traducido y modificado por Martín Sende Pombo. También se empleó ChatGPT 3.5 como asistente de programación.
Fecha de modificación: 06-03-2024
Propósito: Recuperar las parejas letales sintéticas de los genes de la lista dada utilizando el algoritmo DAISY.
Notas: Según el autor puede ejecutarse en MyBinder.
```
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/isb-cgc/Community-Notebooks/HEAD?labpath=SL-Cloud%2FMyBinder_Authentication.ipynb)

Si alguna parte de este análisis se utiliza en una publicación, por favor cite lo siguiente:

Para el algoritmo DAISY:  

Jerby-Arnon, L., Pfetzer, N., Waldman, Y. Y., McGarry, L., James, D., Shanks, E., ... & Gottlieb, E. (2014). Predicting cancer-specific vulnerability via data-driven detection of synthetic lethality. Cell, 158(5), 1199-1209.

Para los datos ómicos de la Cancer Cell Line Encyclopedia (CCLE):

Ghandi, M., Huang, F.W., Jané-Valbuena, J. et al. Next-generation characterization of the Cancer Cell Line Encyclopedia. Nature 569, 503–508 (2019). https://doi.org/10.1038/s41586-019-1186-3

Para datos de cribado funcional obtenidos con CRISPR: 

Robin M. Meyers, Jordan G. Bryan, James M. McFarland, Barbara A. Weir, ... David E. Root, William C. Hahn, Aviad Tsherniak. Computational correction of copy number effect improves specificity of CRISPR-Cas9 essentiality screens in cancer cells. Nature Genetics 2017 October 49:1779–1784. doi:10.1038/ng.3984

Dempster, J. M., Rossen, J., Kazachkova, M., Pan, J., Kugener, G., Root, D. E., & Tsherniak, A. (2019). Extracting Biological Insights from the Project Achilles Genome-Scale CRISPR Screens in Cancer Cell Lines. BioRxiv, 720243.

Para datos de cribado funcional generados mediante ARNhc ("shRNA" en inglés):

James M. McFarland, Zandra V. Ho, Guillaume Kugener, Joshua M. Dempster, Phillip G. Montgomery, Jordan G. Bryan, John M. Krill-Burger, Thomas M. Green, Francisca Vazquez, Jesse S. Boehm, Todd R. Golub, William C. Hahn, David E. Root, Aviad Tsherniak. (2018). Improved estimation of cancer dependencies from large-scale RNAi screens using model-based normalization and data integration. Nature Communications 9, 1. https://doi.org/10.1038/s41467-018-06916-5

Para el ISB Cancer Gateway in the Cloud (ISB-CGC):

Reynolds, S. M., Miller, M., Lee, P., Leinonen, K., Paquette, S. M., Rodebaugh, Z., ... & Shmulevich, I. (2017). The ISB Cancer Genomics Cloud: a flexible cloud-based platform for cancer genomics research. Cancer research, 77(21), e7-e10.

Para los datos del Pan-Cancer Atlas (PanCanAtlas):

Hutter, C., and Zenklusen, J.C. (2018). The Cancer Genome Atlas: Creating Lasting Value beyond Its Data. Cell 173, 283–285.

Este cuaderno es una reimplementación del algoritmo de predicción de pares letales sintéticos DAISY.

Consta de 3 módulos: 

1. Determinación de candidatos a "SL/SDL" mediante coexpresión génica
2. Determinación de candidatos a "SL/SDL" mediante la supervivencia del más apto (SoF)
3. Determinación de candidatos a "SL/SDL" mediante experimentos con CRISPR y ARNhc (FunEx)

* A continuación, los resultados de los tres módulos se agregaron en una lista de pares letales sintéticos ("SL") o letales sintéticos de dosis ("SDL") candidatos.

Parámetros de entrada:
* Tipo de cáncer 
* Los genes cuyas parejas "SL" o "SDL" se buscan

Datos de entrada (disponibles en las tablas de BigQuery):
* Datos de expresión génica 
* Datos de mutación genética
* Datos de variación del número de copias
* Datos de efecto génico (CRISPR)
* Datos de puntuaciones de dependencia génica (ARNhc)

Salida
* Lista de pares "SL" o "SDL" candidatos

Póngase en contacto con Bahar Tercan btercan@systemsbiology.org para cualquier pregunta o información detallada. 

In [1]:
# Por favor, no corra este bloque de código si está ejecutando el cuaderno en el binder asociado de los Community-Notebooks del ISB-CGC.
# Dicho bloque instala las dependencias necesarias. Por favor, descomente (quite la # de) las líneas de código (salvo las correspondientes a bibliotecas ya instaladas) y ejecútelo sólo una vez.

#!pip3 install google.cloud
!pip3 install importlib
#!pip3 install pandas
!pip3 install ipywidgets
!pip3 install numpy
!pip3 install statsmodels
!pip3 install --upgrade google-cloud-bigquery
!pip3 install pyarrow
!pip3 install pandas-gbq 
!pip3 install openpyxl


## 1. Importe bibliotecas de Python

In [3]:
import sys
sys.path.append('../Scripts/') # Para poder utilizar los archivos .py de la carpeta ../Scripts
from google.cloud import bigquery
import importlib
import pandas as pd
import DAISY_operations
importlib.reload(DAISY_operations)
from DAISY_operations import *
import ipywidgets as widgets
import openpyxl

In [ ]:
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

## 2. Autentificación de Google
Ejecutar las celdas de BigQuery en este cuaderno requiere un proyecto de Google Cloud y las instrucciones para crear uno pueden encontrarse en la [documentación de Google](https://cloud.google.com/resource-manager/docs/creating-managing-projects#console). Además, la instancia debe estar autorizada para facturar las consultas del proyecto.
Para más información sobre cómo iniciarse en la nube, consulte ['Quick Start Guide to ISB-CGC'](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/HowToGetStartedonISB-CGC.html). Asimismo, puede localizar métodos de autenticación alternativos en la [documentación de Google](https://googleapis.dev/python/google-api-core/latest/auth.html).

### * Si está ejecutando este cuaderno en un entorno de máquina virtual de Google (Compute Engine, App Engine, Cloud Run, Cloud Functions): 
La autenticación debería "simplemente funcionar".

### * Si la ejecución se realiza en un entorno local: 
La forma más sencilla de autenticarse es emplear el SDK de Google Cloud.

In [ ]:
#Por favor, no corra este bloque de código si ya lo ejecutó en el cuaderno MyBinder.
#Si no está ejecutando el cuaderno en MyBinder, por favor asegúrese de que ha instalado Cloud SDK.
#Consulte la documentación sobre cómo efectuar la instalción en: https://cloud.google.com/sdk/docs/install
#!pip3 install --upgrade google-auth

!gcloud auth application-default login


## 3. Acceda a Google BigQuery con el ID del proyecto
Conexión a Bigquery



In [ ]:
#Sustituya "my-google-project" por el identificador de su proyecto.
project_id='my-google-project'
client = bigquery.Client(project_id)

## 4. Predicción de parejas letales sintéticas utilizando diferentes módulos en DAISY

Hay tres módulos para inferencias de pares letales sintéticos/letales sintéticos de dosis en DAISY:
1. Coexpresión génica por pares. 

2. Supervivencia genómica del más apto (SoF). 

3. Examen funcional (FunEx) basado en ARNhc o CRISPR.
Puede obtener más información en el artículo original: https://www.sciencedirect.com/science/article/pii/S0092867414009775.

En el módulo de coexpresión génica por pares y en el módulo de supervivencia genómica del más apto, utilizaremos los datos del Pan-Cancer Atlas y de la CCLE.<br>
En el módulo de examen funcional, utilizaremos datos CRISPR y ARNhc junto con datos de la CCLE. <br>
Los datos ómicos utilizados proceden de la versión [DepMapPublic_20Q3](https://depmap.org/portal/data_page/?tab=allData), a excepión de los del cribado con ARNi, que emplean la versión 6 de los conjuntos de datos de [DEMETER2](https://depmap.org/R2-D2/).

Los códigos Python necesarios se encuentran en la carpeta ../Scripts/ y son importados al principio.

### 4.0. Parámetros por defecto para DAISY (puede editarlos)
Para la predicción de SDL, sustituya "SL" por "SDL" e "Inactive" por "Overactive" en las siguientes líneas de código.

Para los procedimientos SOF y FuncExamination, input_mutations es un parámetro opcional. Si no desea utilizarlo, puede omitirlo. 

In [ ]:
input_mutations = ['Nonsense_Mutation', 'Frame_Shift_Ins', 'Frame_Shift_Del'] 
# Parámetros por defecto de DAISY para la predicción de "SL":
percentile_threshold = 10 # Usado para decidir si el gen está inactivo.
cn_threshold = -0.3 # Utilizado para decidir si el gen está inactivo.
cor_threshold = 0.5 # Empleado para inferir si dos genes están en una relación de "SL".
p_threshold = 0.05
pval_correction = 'Bonferroni'
fdr_level='gene_level' #Puede ser "gene_level" o "analysis_level".

# Para la predicción de "SDL", los parámetros de DAISY son:
#percentile_threshold = 90
#cn_threshold = 0.3 
#cor_threshold = 0.5
#p_threshold = 0.05


Los tipos de tumor ("tumor type") son los tipos de cáncer del TCGA. Los tipos de cáncer que tienen línes celulares correspondientes se enumeran en el "combobox". Haga clic en el tejido o tejidos en los que desea realizar el análisis.

In [ ]:
TCGA_list=GetTCGASubtypes(client)
TCGA_list = [i for i in TCGA_list if i]

tumor_type = widgets.SelectMultiple(
    options=['pancancer'] + TCGA_list,
    value=[],
    description='Tumor type',
    disabled=False
)
display(tumor_type)

A continuación, tiene que introducir la lista de genes para los que le gustaría encontrar parejas de "SL/SDL". 
Si es posible, emplee sus correspondientes símbolos (abreviaturas) de la HUGO (Organización del Genoma Humano); es decir, los aprobados por el HGNC o HUGO Gene Nomenclature Committee (Comité de Nomenclatura de Genes de HUGO).

In [1]:
# Para utilizar su propia lista de genes, cópiela en la carpeta "Listas_genes" y reemplace el nombre del archivo_actual ("Genes_AmiGO2_ruta_biosintesis_NAD.txt") por el del suyo.
# Función para cambiar el formato de la lista de caracteres
def cambiar_formato_lista(lista):
    # Dividir la lista en líneas y eliminar líneas vacías
    lineas = lista.split('\n')
    lineas = [linea.strip() for linea in lineas if linea.strip()]

    # Devolver la lista en el formato deseado
    return lineas

# Ruta del archivo que contiene la lista en el formato actual
archivo_actual = 'Listas_genes/Genes_AmiGO2_ruta_biosintesis_NAD.txt'

# Lee la lista en el formato actual desde el archivo
lista_actual = ""
with open(archivo_actual, 'r') as archivo:
    lista_actual = archivo.read()

# Cambia el formato de la lista
gene_list = cambiar_formato_lista(lista_actual)

In [6]:
# Opcionalmente puede ejecutarse este bloque de código, para verificar que la lista de genes se ha almacenado correctamente y que tiene el formato adecuado.
# Concretamente, imprime la lista ya con el formato deseado.
print (gene_list)

# Si el resultado es el esperado, ['GENE1', 'GENE2', ... 'GENEn'], entoces puede correr el primer módulo de DAISY.

['NAPRT', 'NADSYN1', 'PTGIS', 'AFMID', 'KMO', 'NMNAT1', 'SLC22A13', 'NNMT', 'PARP16', 'SLC5A8', 'NMRK1', 'HAAO', 'NAMPT', 'ASPDH', 'PARP9', 'NMNAT2', 'PARP10', 'QPRT', 'IDO1', 'NMRK2', 'NMNAT3', 'NAXD', 'KYNU', 'NUDT12', 'IDO2']


In [3]:
# Alternativamente, aunque solo se recomienda si su lista de genes es breve, puede introducir manualmente los genes cambiando el siguiente ejemplo:
gene_list=["BRCA1", "BRCA2", "ARID1A"]

### 4.1. Módulo de coexpresión génica por pares
En el módulo de coexpresión por pares, DAISY realiza inferencias basadas en la suposición de que los pares de genes letales sintéticos desempeñan un papel en procesos biológicos relacionados y se coexpresan. La expresión génica se midió para datos derivados de pacientes del TCGA y de líneas celulares tumorales de la CCLE. La coexpresión por pares se estima a partir de la correlación de Spearman que calculamos entre cada gen de interés (cada elemento de la lista de genes de consulta) y todos los demás genes. Los pares de genes sintéticos letales candidatos son aquellos con un coeficiente de correlación superior a 0,5 y cuyo valor P corregido por Bonferroni es inferior a 0,05 por defecto (estos parámetros pueden ajustarse a continuación). 

#### 4.1.1. Módulo de coexpresión génica por pares para el PanCanAtlas

In [ ]:
coexp_pancancer = CoexpressionAnalysis(client, 'SL', "PanCancerAtlas", gene_list , pval_correction, fdr_level, list(tumor_type.value))
try:
    coex_pan_intermediate_report=coexp_pancancer.loc[(coexp_pancancer['FDR'] < p_threshold)&(coexp_pancancer['Correlation'] > cor_threshold)]
    coexp_pancancer_report=coex_pan_intermediate_report.groupby('Inactive').apply(lambda x: x.sort_values('FDR'))
except:
    coexp_pancancer_report=pd.DataFrame()
    print("No results returned.")
    
coexp_pancancer_report

**Inactive/Overactive** (para "SL" y para "SDL" respectivamente): El nombre del gen procedente de la lista de entrada.

**InactiveDB/OveractiveDB**: El nombre del gen correspondiente en el conjunto de datos (PanCanAtlas).

**SL_Candidate**: El par "SL" candidato del gen en la misma fila. 

**#Samples**: El número de muestras que se han utilizado en el cálculo de la correlación.

**Correlation**: La correlación rho de Spearman en la expresión génica.

**PValue**: El valor p de la correlación de Spearman.

**FDR**: Valor p corregido. 

**Tissue**: El/los tejido/s en el/los que se ha realizado el análisis.

In [ ]:
# Si quiere guardar esta tabla como un archivo de Excel, descomente (quite la #) y ejecute la siguiente línea de código.
#WriteToExcel("CEGP_PCA_SLoSDL_resultados.xlsx", [coexp_pancancer_report], ["Resultados CEGP PCA"])

# El libro de Excel resultante se almacena en la misma carpeta en la que se encuentra este flujo de trabajo; es decir, en este caso en "Example_workflows".

#### 4.1.2. Módulo de coexpresión génica por pares con datos de la CCLE

In [ ]:
coexp_CCLE=CoexpressionAnalysis(client, 'SL', 'CCLE', gene_list, pval_correction, fdr_level, list(tumor_type.value ))
try: 
    coex_ccle_intermediate_report=coexp_CCLE.loc[(coexp_CCLE['FDR'] < p_threshold)&(coexp_CCLE['Correlation'] > cor_threshold)]
    coexp_CCLE_report=coex_ccle_intermediate_report.groupby('Inactive').apply(lambda x: x.sort_values('FDR'))
except:
    coexp_CCLE_report=pd.DataFrame()
    print("No results returned.")
coexp_CCLE_report
    

El mismo análisis/resultado que con el análisis de coexpresión del Pan-Cancer Atlas, excepto porque el conjunto de datos usado procede de la CCLE.

In [ ]:
# Si quiere almacenar esta tabla como un archivo de Excel, descomente (quite la #) y ejecute la siguiente línea de código.
#WriteToExcel("CEGP_CCLE_SLoSDL_resultados.xlsx", [coexp_CCLE_report], ["Resultados CEGP CCLE"])

# El libro de Excel resultante se guarda en la misma carpeta en la que se ubica este flujo de trabajo; es decir, en este caso en "Example_workflows".

### 4.2. Módulo genómico de supervivencia del más apto
El módulo de inferencia de la supervivencia genómica del más apto se basa en la alteración del número de copias del gen en el dominio de búsqueda, dado si el gen de interés es inactivo (o está sobreexpresado) o no. El gen de interés de una muestra se considera inactivo si su expresión es inferior al percentil 10 de todas las muestras y su alteración del número de copias es inferior a -0,3 o si tiene una mutación sin sentido, de desplazamiento de marco o de deleción de marco. El gen de interés de una muestra se considera sobreexpresado si su expresión supera el percentil 90 de todas las muestras y su alteración del número de copias es superior a 0,3 (la sobreexpresión se utiliza en la predicción de pares letales sintéticos de dosis).

Se aplica la prueba de suma de rangos unilateral de Wilcoxon (U de Mann-Whitney) a la alteración del número de copias del par letal sintético candidato de cada gen de interés. El mayor número de copias del par letal sintético candidato para las muestras cuyo gen de interés es inactivo (o está sobreexpresado) se considera un indicador de que los genes se encuentran en una relación letal sintética o letal sintética de dosis. Se devuelven los pares "SL/SDL" cuyo p-valor corregido por Bonferroni es inferior a 0,05. Este procedimiento de inferencia se aplica a los datos del Pan-Cancer Atlas y de la CCLE por separado.

#### 4.2.1. Módulo de supervivencia genómica del más apto para los datos del PanCanAtlas

In [ ]:
sof_pancancer = SurvivalOfFittest(client, 'SL', "PanCancerAtlas", gene_list, percentile_threshold, cn_threshold, pval_correction, fdr_level, list(tumor_type.value), input_mutations)
try:
    sof_pancancer_intermediate_report=sof_pancancer.loc[(sof_pancancer['FDR'] < p_threshold),]                
    sof_pancancer_report=sof_pancancer_intermediate_report.groupby('Inactive').apply(lambda x: x.sort_values('FDR'))
except:
    sof_pancancer_report=pd.DataFrame()
    print("No results returned.") 
sof_pancancer_report

**Inactive/Overactive** (para "SL" y para "SDL" respectivamente): El nombre del gen procedente de la lista de entrada.

**InactiveDB/OveractiveDB**: El nombre del gen correspondiente en el conjunto de datos (Pan-Cancer Atlas).

**SL_Candidate**: El par "SL" candidato del gen en la misma fila. 

**#InactiveSamples**: El número de muestras inactivas.

**#Samples**: El número total de muestras sobre las que se realiza el análisis.

**PValue**: Los valores p para la prueba unilateral no emparejada de Wilcoxon aplicada a los datos de alteración del número de copias somáticas.

**FDR**: P-valor corregido.

**Tissue**: El/los tejido/s en el/los que se ha efectuado el análisis.

In [ ]:
# Si quiere guardar esta tabla como un archivo de Excel, descomente (quite la #) y ejecute la siguiente línea de código.
#WriteToExcel("SoF_PCA_SLoSDL_resultados.xlsx", [sof_pancancer_report], ["Resultados SoF PCA"])

# El libro de Excel resultante se almacena en la misma carpeta en la que se localiza este flujo de trabajo; es decir, en este caso en "Example_workflows".

#### 4.2.2. Genomic survival of fittest module on CCLE data

In [ ]:
sof_CCLE = SurvivalOfFittest(client, 'SL', "CCLE", gene_list,  percentile_threshold, cn_threshold, pval_correction, fdr_level, list(tumor_type.value), input_mutations)
try: 
    sof_ccle_intermediate_report=sof_CCLE.loc[(sof_CCLE['FDR'] < p_threshold),]
    sof_ccle_report=sof_ccle_intermediate_report.groupby('Inactive').apply(lambda x: x.sort_values('FDR'))
except:
    sof_ccle_report=pd.DataFrame()
    print("No results returned.")
sof_ccle_report

El mismo análisis/resultado que con el análisis de superivivencia del más apto apliacado al Pan-Cancer Atlas, excepto porque el conjunto de datos empleado proviene de la CCLE.

In [ ]:
# Si quiere almacenar esta tabla como un archivo de Excel, descomente (quite la #) y ejecute la siguiente línea de código.
#WriteToExcel("SoF_CCLE_SLoSDL_resultados.xlsx", [sof_ccle_report], ["Resultados SoF CCLE"])

# El libro de Excel resultante se guarda en la misma carpeta en la que se ecuentra este flujo de trabajo; es decir, en este caso en "Example_workflows".

### 4.3. Módulo de inferencia de examen funcional
El fundamento del módulo de inferencia de examen funcional es que si la pareja letal sintética de un gen está inactiva en una muestra dada, la inactivación posterior de ese gen será letal. Por lo tanto, para un gen de interés, primero definimos dos grupos para la prueba, uno en el que el gen estaba inactivo y otro en el que no lo estaba. A continuación, realizamos una prueba de suma de rangos de Wilcoxon unilateral (U de Mann-Whitney) sobre la sensibilidad "knockdown"/"knockout" de los pares letales sintéticos candidatos de interés. Una menor viabilidad asociada a una mayor sensibilidad al silenciamiento/bloqueo génico es un indicador de una posible interacción de letalidad sintética ("SLI"). Se devuelven las parejas letales sintéticas para las que el valor P del resultado de la prueba es inferior a 0,05. Este procedimiento de inferencia se aplica a las puntuaciones de dependencia génica o de efecto génico para los conjuntos de datos de ARNhc y CRISPR por separado.

#### 4.3.1. Módulo de inferencia de exámenes funcionales basado en CRISPR

In [ ]:
crispr_result = FunctionalExamination(client,'SL', "CRISPR", gene_list, percentile_threshold, 
                                      cn_threshold, pval_correction,  fdr_level, list(tumor_type.value), input_mutations )
try:
    crispr_intermediate_report=crispr_result.loc[(crispr_result['PValue'] < p_threshold),]
    crispr_report=crispr_intermediate_report.groupby('Inactive').apply(lambda x: x.sort_values('PValue'))
except:
    crispr_report=pd.DataFrame()
    print("No results returned.")
crispr_report   

**Inactive/Overactive** (para "SL" y para "SDL" respectivamente): El nombre del gen procedente de la lista de entrada.

**InactiveDB/OveractiveDB**: El nombre del gen correspondiente en el conjunto de datos (Pan-Cancer Atlas).

**SL_Candidate**: El par SL candidato del gen en la misma fila.  

**#InactiveSamples**: El número de muestras inactivas.

**#Samples**: El número total de muestras sobre las que se lleva a cabo el análisis. 

**PValue**: Los p-valores para la prueba unilateral pareada de Wilcoxon sobre los datos de esencialidad del gen CRISPR.

**FDR**: Valor p corregido.

**Tissue**: El/los tejido/s en el/los que se ha realizado el análisis.

In [ ]:
# Si quiere guardar esta tabla como un archivo de Excel, descomente (quite la #) y ejecute la siguiente línea de código.
#WriteToExcel("FunEx_CRISPR_SLoSDL_resultados.xlsx", [crispr_report], ["Resultados FunEx CRISPR"])

# El libro de Excel resultante se almacena en la misma carpeta en la que se ubica este flujo de trabajo; es decir, en este caso en "Example_workflows".

#### 4.3.2. Módulo de inferencia de exámenes funcionales basado en ARNhc

In [ ]:
shRNA_result = FunctionalExamination(client, 'SL', "shRNA", gene_list , percentile_threshold, \
                                     cn_threshold, pval_correction,  fdr_level, list(tumor_type.value),input_mutations)
try:
    shRNA_intermediate_report=shRNA_result.loc[(shRNA_result['PValue'] < p_threshold),]
    shRNA_report=shRNA_intermediate_report.groupby('Inactive').apply(lambda x: x.sort_values('PValue'))
    
except:
    shRNA_report=pd.DataFrame()
    print("No results returned.")
shRNA_report

El mismo análisis/resultado que con el análisis de Examen Funcional CRISPR, excepto que se usa el conjunto de datos "shRNA- gene dependency".

In [ ]:
# Si quiere almacenar esta tabla como un archivo de Excel, descomente (quite la #) y ejecute la siguiente línea de código.
#WriteToExcel("FunEx_ARNhc_SLoSDL_resultados.xlsx", [shRNA_report], ["Resultados FunEx ARNhc"])

# El libro de Excel resultante se guarda en la misma carpeta en la que se localiza este flujo de trabajo; es decir, en este caso en "Example_workflows".

## 5. Integración de resultados

### 5.1. Integración de los resultados de coexpresión génica por pares del PanCanAtlas y de la CCLE
Se efectúa la unión de los resultados del Pan-Cancer Atlas y de la CCLE.

In [ ]:
try:
    coexpression_result = UnionResults([coexp_pancancer_report, coexp_CCLE_report],'SL', ['FDR', 'FDR'],  list(tumor_type.value))
    coexpression_result=coexpression_result.sort_values('Inactive')
except:
    coexpression_result=pd.DataFrame()
    print("No Result From Pairwise Co-expression Inference Procedure")
    
coexpression_result

**Inactive**: El gen de entrada del usuario.

**SL_Candidate/SDL_Candidate**: El par "SL" o "SDL" candidato. 

**p values or FDRs**: El valor p agregado o FDR obtenido con el método de Fisher.

**Tissue**: El/los tejido/s en el/los que se ha realizado el análisis.

In [ ]:
# Con este código puede obtener un libro de Excel con solo los resultados de este método de inferencia. Si no le interesa, puede omitirlo.
WriteToExcel("CEGP_SLoSDL_resultados.xlsx", [coexpression_result], ["Resultados CEGP"])

# Dicho archivo de Excel resultante se almacena en la misma carpeta en la que se ecuentra este flujo de trabajo; es decir, en este caso en "Example_workflows".

### 5.2. Integración de los resultados de supervivencia del más apto del PanCanAtlas y de la CCLE
Se realiza la unión de los resultados del Pan-Cancer Atlas y de la CCLE. 

In [ ]:
try:
    sof_result = UnionResults([sof_ccle_report, sof_pancancer_report],  'SL', ['FDR', 'FDR'], list(tumor_type.value))
    sof_result=sof_result.sort_values('Inactive')
except:
    sof_result=pd.DataFrame()
    print("No Result From Survival of Fittest Inference Procedure")
sof_result    

**Inactive**: El nombre del gen inactivo.

**SL_Candidate/SDL_Candidate**: La pareja "SL" o "SDL" candidata. 

**p values or FDRs**: El p-valor agregado o FDR obtenido con la prueba de Fisher.

**Tissue**: El/los tejido/s en el/los que se llevó a cabo el análisis.

In [ ]:
# Con este código puede obtener un libro de Excel con solo los resultados de este método de inferencia. 
# Si no le interesa, puede saltarlo.
WriteToExcel("SoF_SLoSDL_resultados.xlsx", [sof_result], ["Resultados SoF"])

# Dicho archivo de Excel resultante se guarda en la misma carpeta en la que se ubica este flujo de trabajo; es decir, en este caso en "Example_workflows".

### 5.3. Integración del módulo de inferencia de examen funcional basado CRISPR y en ARNhc.
Informamos de la unión de los resultados de los conjuntos de datos basados en CRISPR y ARNhc. 

In [ ]:
try:
    functional_screening_result = UnionResults([crispr_report, shRNA_report],'SL', ['PValue', 'PValue'], list(tumor_type.value))
    functional_screening_result=functional_screening_result.sort_values('Inactive')
    
except:
    functional_screening_result=pd.DataFrame()
    print("No Result From Functional Examination Inference Procedure")
functional_screening_result    

**Inactive**: El nombre del gen inactivo.

**SL_Candidate/SDL_Candidate**: El par "SL" o "SDL" candidato.

**p values or FDRs**: El valor p agregado o FDR obtenido con la prueba de Fisher.

**Tissue**: El/los tejido/s en el/los que se efectuó el análisis.

In [ ]:
# Con este código puede obtener un libro de Excel con solo los resultados de este método de inferencia. 
# Si no le interesa, puede omitirlo.
WriteToExcel("FunEx_SLoSDL_resultados.xlsx", [functional_screening_result], ["Resultados FunEx"])

# Dicho archivo de Excel resultante se almacena en la misma carpeta en la que se localiza este flujo de trabajo; es decir, en este caso en "Example_workflows".

### 5.4. Fusión de los resultados de los tres procedimientos de inferencia
La intersección de pares de "SL" de los distintos procedimientos de inferencia compone la lista final.

In [ ]:
try:
    all_merged_results = MergeResults([coexpression_result, sof_result, functional_screening_result], 'SL',  list(tumor_type.value))
    all_merged_results=all_merged_results.sort_values('Inactive')
except:
    all_merged_results=pd.DataFrame()
    print("No results found")
all_merged_results

**Inactive/Overactive**: El gen de entrada del usuario.


**SL/SDL candidate**: La pareja "SL/SDL" inferida del gen en la misma fila.

Los resultados también pueden guardarse en un archivo de Excel, el cual aparecerá en la misma carpeta en la que se encuentra este flujo de trabajo; es decir, en este caso en "Example_workflows".

In [ ]:
WriteToExcel("DAISY_SLoSDL_resultados.xlsx", [all_merged_results], ["Resultados finales"])